## SPIKE_SARS2

+ wild type sequencehas 1273 aa
+ aim: generate a series of variants that will accumulate (at randomly chosen positions) aa substitutions for each type of aa
+ abt 2.5 million sequences are cretaed for 100 repetitions (1273 x 20 x 100)

In [5]:
import random
import os
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import timeit

MissingPythonDependencyError: Please install numpy if you want to use Bio.Align. See http://www.numpy.org/

In [4]:
# SARS-CoV-2 spike protein
spike_sars2 = SeqIO.read('P0DTC2.fasta', 'fasta')
spike_sars2_str = str(spike_sars2.seq)

NameError: name 'SeqIO' is not defined

In [34]:
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'

### percentage distribution of aa in spike_sars2

In [35]:
# count of aa
x = ProteinAnalysis(str(spike_sars2.seq))
aa_counts = x.count_amino_acids()

#length of spike protein
total_aa = len(spike_sars2_str)

# percentage of aa
aa_percentages = {aa: (count / total_aa) * 100 for aa, count in aa_counts.items()}

# table
print("{:<10}| {:}".format("Amino Acid", "Counts (Percentage)"))
print("=" * 26)

for aa, count in aa_counts.items():
    percentage = aa_percentages[aa]
    print("{:<10}| {:} ({:.2f}%)".format(aa, count, percentage))
 

Amino Acid| Counts (Percentage)
A         | 79 (6.21%)
C         | 40 (3.14%)
D         | 62 (4.87%)
E         | 48 (3.77%)
F         | 77 (6.05%)
G         | 82 (6.44%)
H         | 17 (1.34%)
I         | 76 (5.97%)
K         | 61 (4.79%)
L         | 108 (8.48%)
M         | 14 (1.10%)
N         | 88 (6.91%)
P         | 58 (4.56%)
Q         | 62 (4.87%)
R         | 42 (3.30%)
S         | 99 (7.78%)
T         | 97 (7.62%)
V         | 97 (7.62%)
W         | 12 (0.94%)
Y         | 54 (4.24%)


### calculate variants of wildtype sequence of spike_sars2 towards all 20 homopolymers

In [97]:
def sequence_generator(input_sequence = spike_sars2_str, reps = 1, output_directory = "aa_trajectories"):

    # cretae output directory
    for rep in range(1, reps + 1):
        rep_output_directory = os.path.join(output_directory, str(rep))
        os.makedirs(rep_output_directory, exist_ok=True)

        # Loop through each aa
        for amino_acid in amino_acids:
            trajectories_for_amino_acid = []
            for _ in range(reps):

                # Create a copy of the input sequence with the chosen amino acid
                input_sequence_copy = input_sequence

                # Initialize a set to store unique sequences in this trajectory
                sequences_in_trajectory = []

                # Continue until the sequence is a homopolymer of the current amino acid
                while any(aa != amino_acid for aa in input_sequence_copy):
                    # Randomly choose a position to mutate
                    position_to_mutate = random.randint(0, len(input_sequence_copy) - 1)

                    if input_sequence_copy[position_to_mutate] != amino_acid:
                        # takes a substring from the beginning of input_sequence_copy up until the position to mutate and a substring after
                        input_sequence_copy = input_sequence_copy[:position_to_mutate] + amino_acid + input_sequence_copy[position_to_mutate + 1:]
                        sequences_in_trajectory.append(input_sequence_copy)

                # Append the unique sequences to the list of trajectories for this amino acid
                trajectories_for_amino_acid.extend(sequences_in_trajectory)

            # Create SeqRecord objects and save them to a FASTA file
            output_filename = f"{rep_output_directory}/{amino_acid}_trajectories.fasta"
            seq_records = [SeqRecord(Seq(seq), id=f"{amino_acid}_trajectory_{i+1}", description=f"Generated variant {i+1} for trajectory towards homopolymer {amino_acid}") for i, seq in enumerate(trajectories_for_amino_acid)]

            with open(output_filename, "w") as output_file:
                SeqIO.write(seq_records, output_file, "fasta")

        print(f"Generated trajectories for all 20 homopolymers in '{rep_output_directory}' directory.")

###  check if the correct amount of sequences was produced

In [111]:
def control_sequences(fasta_dir = "aa_trajectories", print_number_seq = True, input_seq = spike_sars2_str, reps=1):

    total_sequence_count = 0
    
    for root, _, files in os.walk(fasta_dir):
        for filename in files:
            file_path = os.path.join(root, filename)

            if filename.endswith(".fasta"):
                # Count sequences in the current file
                with open(file_path, "r") as fasta_file:
                    total_sequence_count += len(list(SeqIO.parse(fasta_file, "fasta")))
    

    x = ProteinAnalysis(input_seq)
    aa_counts = x.count_amino_acids()
    n_sequences = 0
    for aa, count in aa_counts.items():

        temp_n_sequences = 0
        temp_n_sequences += total_aa - count
        n_sequences += temp_n_sequences

    if print_number_seq == True:
        print(f"Total number of sequences: {total_sequence_count}")
        print(f"Total number of possible sequences: {n_sequences*reps}")

    #check if correct amount of variants have been produced
    if total_sequence_count == n_sequences: print(True)

In [113]:
sequence_generator(spike_sars2_str, 1,"aa_trajectories")
print('\n')
control_sequences(fasta_dir = "aa_trajectories", print_number_seq = True, input_seq = spike_sars2_str,reps = 1)

Generated trajectories for all 20 homopolymers in 'aa_trajectories\1' directory.


Total number of sequences: 24187
Total number of possible sequences: 24187
True


### time

In [80]:
elapsed_time = timeit.timeit(sequence_generator, number=1)

print(f"Elapsed time: {elapsed_time} seconds")

Generated trajectories for all 20 homopolymers in 'aa_trajectories\1' directory.
Elapsed time: 4.2457524000001285 seconds
